In [1]:
# %%
%pip install datasets
%pip install peft
%pip install -U bitsandbytes


In [2]:
# %%
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq
)
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import psutil
import os
import numpy as np


In [3]:
# %%
def format_example(example):
    """Format a single example for fine-tuning."""
    return f"""Problem: {example['markdown_description']}\n\nSolution: {example['canonical_solution']}"""

def get_memory_usage():
    """Get current memory usage in MB."""
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024


In [4]:
# %%
def prepare_dataset():
    """Prepare the Effibench dataset for fine-tuning."""
    dataset = load_dataset("DONG19/EffiBench")
    print("Sample record:", dataset['train'][0])  # Inspect a single record

    def format_function(examples):
        inputs = [f"Problem: {md}" for md in examples["markdown_description"]]
        outputs = [f"Solution: {cs}" for cs in examples["canonical_solution"]]
        return {"input_text": inputs, "output_text": outputs}

    # Tokenization function
    def tokenize_function(examples):
        inputs = tokenizer(
            examples["input_text"], padding="max_length", truncation=True, max_length=1024
        )
        outputs = tokenizer(
            examples["output_text"], padding="max_length", truncation=True, max_length=1024
        )
        inputs["labels"] = outputs["input_ids"]
        return inputs

    # Format the dataset
    formatted_dataset = dataset.map(
        format_function,
        remove_columns=dataset["train"].column_names,
        batched=True  # Process in batches
    )

    # Tokenize the dataset
    tokenized_dataset = formatted_dataset.map(
        tokenize_function,
        batched=True
    )

    return tokenized_dataset


In [5]:
# %%
# Initialize memory tracking
initial_memory = get_memory_usage()
memory_measurements = []


In [6]:
# %%
# Model configuration
model_name = "NousResearch/Llama-2-7b-hf"  # Using 7B parameter model as it's more consumer-friendly

# QLoRA configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)
model = prepare_model_for_kbit_training(model)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
# %%
# LoRA configuration
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ2SEQ_LM"
)

# Apply LoRA
model = get_peft_model(model, lora_config)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token


In [8]:
# %%
# Prepare dataset
dataset = prepare_dataset()


Sample record: {'problem_idx': 3, 'task_name': 'Longest Substring Without Repeating Characters', 'description': '\n\n<p>Given a string <code>s</code>, find the length of the <strong>longest</strong> <span data-keyword="substring-nonempty"><strong>substring</strong></span> without repeating characters.</p>\n\n<p>&nbsp;</p>\n<p><strong class="example">Example 1:</strong></p>\n\n<pre>\n<strong>Input:</strong> s = &quot;abcabcbb&quot;\n<strong>Output:</strong> 3\n<strong>Explanation:</strong> The answer is &quot;abc&quot;, with the length of 3.\n</pre>\n\n<p><strong class="example">Example 2:</strong></p>\n\n<pre>\n<strong>Input:</strong> s = &quot;bbbbb&quot;\n<strong>Output:</strong> 1\n<strong>Explanation:</strong> The answer is &quot;b&quot;, with the length of 1.\n</pre>\n\n<p><strong class="example">Example 3:</strong></p>\n\n<pre>\n<strong>Input:</strong> s = &quot;pwwkew&quot;\n<strong>Output:</strong> 3\n<strong>Explanation:</strong> The answer is &quot;wke&quot;, with the length 

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [9]:
# %%
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    warmup_steps=100,
    logging_steps=100,
    learning_rate=2e-4,
    fp16=True,
    save_strategy="epoch",
    # predict_with_generate=True  # Required for seq2seq tasks
)


In [10]:
# %%
# Data collator for seq2seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    data_collator=data_collator,
)


In [11]:
# %%
# Record pre-training memory
pre_training_memory = get_memory_usage()
memory_measurements.append(("Pre-training", pre_training_memory))


In [12]:
# %%
# Train the model
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: arnavjoshi007 (arnavjoshi007-northeastern-university). Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss


TrainOutput(global_step=62, training_loss=5.1577911376953125, metrics={'train_runtime': 2990.905, 'train_samples_per_second': 0.334, 'train_steps_per_second': 0.021, 'total_flos': 4.032188231398195e+16, 'train_loss': 5.1577911376953125, 'epoch': 0.992})

In [13]:
# %%
# Record post-training memory
post_training_memory = get_memory_usage()
memory_measurements.append(("Post-training", post_training_memory))

# Print memory usage statistics
print("\nMemory Usage Statistics:")
print(f"Initial Memory Usage: {initial_memory:.2f} MB")
for stage, memory in memory_measurements:
    print(f"{stage} Memory Usage: {memory:.2f} MB")



Memory Usage Statistics:
Initial Memory Usage: 1040.91 MB
Pre-training Memory Usage: 1541.10 MB
Post-training Memory Usage: 1869.52 MB


In [14]:
# Save the fine-tuned model and tokenizer
model_save_path = "./fine_tuned_model"
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

print(f"Model and tokenizer saved to {model_save_path}")


Model and tokenizer saved to ./fine_tuned_model


In [15]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
# Copy model and tokenizer from saved folder to Google Drive
!cp -r "./fine_tuned_model" "/content/drive/MyDrive/CS 6158/CS6158 Research Project/fine_tuned_model"

In [17]:
def perform_inference(memory_model, memory_tokenizer, test_dataset, num_samples=5):
    """Perform inference using the model in memory."""
    # Select a few examples from the test set
    test_samples = test_dataset["test"].select(range(num_samples))

    for idx, example in enumerate(test_samples):
        input_text = f"Problem: {example['markdown_description']}"
        # Tokenize the input
        inputs = memory_tokenizer(input_text, return_tensors="pt", truncation=True, padding="max_length", max_length=256).to(memory_model.device)

        # Generate prediction
        outputs = memory_model.generate(**inputs, max_length=256, num_beams=5, early_stopping=True)
        prediction = memory_tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Display result
        print(f"Test Sample {idx + 1}:")
        print(f"Input: {input_text}")
        print(f"Generated Solution: {prediction}")
        print("-" * 80)

# Perform inference using the model in memory
perform_inference(model, tokenizer, dataset)

KeyError: 'test'